# Introduction
This document contains the optimization methods developed for the TFG (Final Degree Project), in three variations:
1. Coefficient-based optimization, focusing on minimizing energy waste.
2. User-specific energy optimization, aiming to minimize the energy wasted to the grid while ensuring each user receives the percentage of energy they have contracted from the system.
3. Euro-based optimization, minimizing the monetary value wasted to the grid. This is analogous to the previous method but ensures that each user receives the proportion of the total energy cost that they have contracted.

### Quick Guide to Library Installation
Assuming a Linux system with Python3 installed:
1. Install pip3:
    ```sh
    sudo apt install python3-pip
    ```
2. Install the libraries:
    ```sh
    pip install pandas numpy scipy
    ```
3. Install Jupyter:
    ```sh
    pip install jupyter
    ```

In [34]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import time

# Import the energy_data_processing.py file
import energy_data_processing as edp

print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)
print("SciPy version:", scipy.__version__)


Pandas version: 1.5.3
NumPy version: 1.26.4
SciPy version: 1.13.0


# Data Generation
The following code processes raw data and generates sample data with the specified characteristics in the `output_files` folder. You can modify:

1. The `file_number` parameter. It refers to the raw files. Currently, only `file1` exists in the folder (though there are 6 in total), so do not modify it for now. We are only considering the data from this file.
2. The number of days. Keep it fixed at 30, as the generation data is only available for 30 days and changing this will cause errors.
3. The number of users. Select between 3 and 8; more users may lead to excessively long execution times.
4. The `selection_type` parameter. Choose `low` for users with low consumption deviation, `top` for users with high consumption deviation, or `random` for random selection.
5. The `optimization1` parameter. Set to `True` to optimize for minimizing energy waste.
6. The `optimization2` parameter. Set to `True` to optimize for minimizing monetary waste (euros).
7. The `max_iter` parameter. Set the maximum number of iterations. For 4 to 8 users, about 10 iterations are sufficient.
8. The `system_coefficients` parameter. These are the participation coefficients for each user. If left empty, they are assumed to be constant. Otherwise, they should be filled in such a way that their sum equals one; otherwise, an error will be thrown.

Running the following code will generate a file in the `output_files` folder with the given configuration name.

In [15]:
file_number = '1'  # DO NOT modify
n_days = 30  # DO NOT modify

# Modify the following values
n_users = 3                # <--- modify if needed
selection_type = 'random'  # modify if needed (low, top, random)

optimization1 = True       # <--- modify if needed
optimization2 = True      # <--- modify if needed

max_iter = 1
# ex for 4 users: system_coefficients = [0.2, 0.2, 0.2, 0.4]
# leave the list empty if you want to use equidistant coefficients
system_coefficients = []  # <--- modify if needed

# Call the main function of energy_data_processing.py
edp.main(file_number, n_users, selection_type)

Los datos han sido guardados en 'final_data_File1_Users3_Random.csv'


## Data Reading
In this section, we load the data. With the previous configuration, the associated data will be loaded automatically.

Throughout the notebook, different important variables are used:
- `E` represents the consumption matrix with `n_rows` users and `n_columns` hours. In all cases, we are dealing with a month, i.e., 720 columns (hours).
- `total_e` is an array with the generation data for a full month, with 720 positions (hours).

In [16]:
if selection_type == 'top':
    file_name = 'final_data_File' + str(file_number) + '_Users' + str(n_users) +'_Top.csv'
elif selection_type == 'low':
    file_name = 'final_data_File' + str(file_number) + '_Users' + str(n_users) +'_Low.csv'
elif selection_type == 'random':
    file_name = 'final_data_File' + str(file_number) + '_Users' + str(n_users) +'_Random.csv'
else:
    raise ValueError("Tipo de selección no válido: elija 'top', 'low' o 'random'.")

file_name = 'output_files/' + file_name
E = pd.read_csv(file_name, index_col=0)

E = pd.DataFrame(E)
print("The dimensions of the matrix are: ", E.shape)


The dimensions of the matrix are:  (3, 720)


Now we load the generation data, which are currently fixed. The data is in Wh, so we divide by 1000 to convert it to kWh.

In [17]:
generation = pd.read_csv('source_data/generacion.csv')
generation = generation.iloc[:,1]
gen_array = generation.values
total_e = gen_array
total_e = [round(i/1000, 2) for i in total_e]  # convert to kWh
total_e = np.array(total_e)

t = n_users // 4  # uses to adjust the total energy
if t == 0:
    t = 1
total_e = total_e*t

# Variable Processing
Once the data is loaded, it needs to be processed. This processing involves removing rows based on certain criteria to have fewer coefficients to optimize.

First, we remove hours with zero production (at night), as optimizing in this case makes no sense. 
Additionally, we remove instances where the generation for that day exceeds the total consumption of all users combined. In this case, the coefficient will be the same for everyone.

We remove these instances from `total_e` and `E`. The removed indices are stored in the variable `indices_nulos`, which will be useful in the future if we need to reconstruct the complete solution.

Thus, the cleaned data we will use from now on are `total_e_no_nulo` and `E_no_nulo`.

In [18]:
null_indices = np.where(total_e == 0)[0]  # Indices to reconstruct the matrix
print("Null indices: ", null_indices.size)

exceeding_indices = np.where(total_e >= E.sum(axis=0))[0]
print("Indices exceeding: ", exceeding_indices.size)

difference = 0
total_e_non_null = total_e.copy()

for i in exceeding_indices:
    difference += total_e_non_null[i] - E.sum(axis=0)[i]
    total_e_non_null[i] = E.sum(axis=0)[i]

total_e_non_null = np.delete(total_e_non_null, null_indices)  # Null values from total_e are removed

print("The number of kWh that have been eliminated because spillage is inevitable is: ", difference, "kWh")

E_non_null = E.drop(E.columns[null_indices], axis=1)  # Columns corresponding to null values in total_e are removed
print("The new dimensions of the matrix are: ", E_non_null.shape)


Null indices:  265
Indices exceeding:  34
The number of kWh that have been eliminated because spillage is inevitable is:  38.67500000000001 kWh
The new dimensions of the matrix are:  (3, 455)


Some relevant data from our sample are:

In [19]:
n_rows, n_cols = E_non_null.shape
nTotHour = total_e_non_null.shape[0]
print("  In total, there are ", nTotHour, " hours (columns)")
print("  In total, there are ", n_rows, " users (rows)")

print("  The sum of the energy generation is: ", total_e_non_null.sum())
# The demand of each user is the sum of each row, i.e., the sum of each user at each hour
print("  The sum of demand for each user is: ", np.sum(E_non_null, axis=1))
print("  Each user should end up consuming ", total_e_non_null.sum()/n_rows, " kWh in an equitable case")


  In total, there are  455  hours (columns)
  In total, there are  3  users (rows)
  The sum of the energy generation is:  967.7149999999999
  The sum of demand for each user is:  UserId
1699     513.714
1181    1298.585
1200     638.692
dtype: float64
  Each user should end up consuming  322.57166666666666  kWh in an equitable case


Ahora estamos en condiciones de comenzar a optimizar los coeficientes.

# Optimization 1
The distribution can be equitable or not, meaning users can have the same percentage of the system, or some can have more than others.

To achieve this, we sum the total generation, and the constraint for each row should be this total divided by the number of users (in the equitable case), or in a general case, multiplied by the coefficient they have contracted.

The new optimization problem is:

$$ \min \sum_{j=1}^{nCols}\sum_{i=1}^{nUser} \max(0, c_{ij} \cdot {totalEnergy_j} - E_{ij})$$

With the constraints:
$$\sum_{i=1}^{nUser} c_{ij} = 1, \forall j \in nCols$$
$$\sum_{j=1}^{nCols} totalEnergy_j \cdot c_{ij} = consumption_i, \forall i \in nUser$$

But, what is consumption? Consumption is an array with the users' consumption in the solar panel system, i.e., how many kWh each user should consume by the end of the month. This consumption is calculated by multiplying each user's system coefficient by the total generation. However, an important case to consider is when a user has contracted more energy than they consume that month. In that case, the user's consumption is reduced to their actual consumption, aiming to avoid energy waste to the grid, and the difference is redistributed among the other users so they can utilize it.

Next, we configure the problem.

In [20]:
n_rows, n_cols = E_non_null.shape
nUser = n_rows

if system_coefficients == []:
    print("Assuming equitable consumption coefficients")
    system_coefficients = np.ones(nUser) / nUser
else:
    # Verify that the sum of the coefficients equals 1 and matches the number of users
    if np.isclose(np.sum(system_coefficients), 1, rtol=1e-8) and len(system_coefficients) == nUser:
        print("System consumption coefficients: ", system_coefficients)
        system_coefficients = np.array(system_coefficients)
    else:
        print("Error: System consumption coefficients must sum to 1 and match the number of users")
        exit()
        
# Fill initial_guess with the system consumption coefficients, a matrix of n_rows x n_cols, with each column equal to system_coefficients
initial_guess = np.zeros((n_rows, n_cols))
for j in range(n_cols):
    initial_guess[:, j] = system_coefficients
print("initial_guess: ", initial_guess.size)



Assuming equitable consumption coefficients
initial_guess:  1365


Now, we need to adjust the row sum constraint. There may be cases where a user's total consumption is less than their allocated share. Therefore, we calculate the maximum energy each user could obtain from the system. If some user(s) consume less energy than their share, we distribute the remaining energy equitably among the other users to avoid unnecessary energy waste to the grid.

In [21]:
print("The sum of what each user consumes is: ", np.sum(E_non_null, axis=1)) 
row_sum = np.sum(total_e_non_null) * system_coefficients
row_sum = np.array(row_sum)

print("The sum of demand for each user is: ", row_sum)

consumption = np.zeros(n_rows)
difference = 0
count = 0
for i in range(n_rows):
    if row_sum[i] >= np.sum(E_non_null.iloc[i, :]):
        # Save the difference
        consumption[i] = np.sum(E_non_null.iloc[i, :])
        difference += row_sum[i] - np.sum(E_non_null.iloc[i, :])
        count += 1

# Distribute difference among other users
for i in range(n_rows):
    if consumption[i] == 0:
        consumption[i] = row_sum[i] + difference / (n_rows - count)

print("The total consumption sum is: ", np.sum(consumption))
print("Consumption: ", consumption)
print("The total energy generation sum is: ", np.sum(row_sum))




The sum of what each user consumes is:  UserId
1699     513.714
1181    1298.585
1200     638.692
dtype: float64
The sum of demand for each user is:  [322.57166667 322.57166667 322.57166667]
The total consumption sum is:  967.7149999999998
Consumption:  [322.57166667 322.57166667 322.57166667]
The total energy generation sum is:  967.7149999999998


Now, we define the new objective functions and constraints. Note that in constraint 2, the previously calculated consumption is subtracted.

In [22]:
def objective_function2(c):
    c_matrix = c.reshape((n_rows, n_cols))
    total = 0
    for j in range(n_cols):
        for i in range(n_rows):
            total += max(0, c_matrix[i, j] * total_e_non_null[j] - E_non_null.iloc[i, j])
    return total

def equality_constraint2_1(c):
    return np.sum(c.reshape((n_rows, n_cols)), axis=0) -1
def equality_constraint2_2(c):
    return np.sum((total_e_non_null * c.reshape((n_rows, n_cols))), axis=1) - consumption

## 1.1 Optimization using SLSQP
Now, a solution using SLSQP is proposed. This solution may take time to converge or might not converge at all. Therefore, it is advisable to adjust the maximum number of iterations as necessary. For small problems with fewer than 20 users, a very good solution can be obtained with few iterations. However, larger problems may require more iterations and more computation time.

Next, we configure the method from the library.

In [23]:
initial_waste_equal_coeffs = objective_function2(initial_guess)
print("Initial waste with equitable coefficients:", initial_waste_equal_coeffs)
print("Each user must consume", consumption, "kWh")

bounds = [(0, 1)] * (n_rows * n_cols)

constraints = ({'type': 'eq', 'fun': equality_constraint2_1},
               {'type': 'eq', 'fun': equality_constraint2_2})

def callback_function(xk):
    if (callback_function.iteration == 0):
        print("The total energy generated is: ", np.sum(total_e_non_null))
        print("The total energy corresponding to each user is: ", consumption)
        
    print("Iteration:", callback_function.iteration)
    print("Objective function value:", objective_function2(xk))
    # print("Decision variables:", xk)
    # print("Constraint 1:", equality_constraint1(xk))
    # print("Constraint 2:", equality_constraint2(xk))
    print("--------------------------")
    callback_function.iteration += 1

callback_function.iteration = 0
# Initialize the coefficients randomly
initial_guess2 = np.random.rand(n_rows, n_cols)

# Normalize each column to sum to 1
initial_guess2 /= np.sum(initial_guess2, axis=0)

# Convert the matrix into a one-dimensional array
initial_guess2 = initial_guess2.flatten()

Initial waste with equitable coefficients: 142.57200000000006
Each user must consume [322.57166667 322.57166667 322.57166667] kWh


In [24]:
if optimization1 == True:
    print("Method set up with: ", max_iter, " iterations")
    start_time = time.time()

    res = minimize(objective_function2, initial_guess2, bounds=bounds, constraints=constraints, method='SLSQP', callback=callback_function, options={'disp': True, 'maxiter': max_iter})
    
    end_time = time.time()
    execution_time = end_time - start_time

    c_optimal = res.x.reshape((n_rows, n_cols))
    
    # Verify that constraints are met
    print("Constraint 1:", equality_constraint2_1(c_optimal))
    print("Constraint 2:", equality_constraint2_2(c_optimal))

    coefficients_df = pd.DataFrame(c_optimal, columns=[f'c{j}' for j in range(1, n_cols + 1)])
    coefficients_df.to_csv('coefficients/optimization1_SLSQP.csv', index=False)

    final_waste_SLSQP = objective_function2(c_optimal)
    time_SLSQP2 = execution_time

    print("---------------------------------------------------------")
    print("Initial waste with equitable coefficients:", initial_waste_equal_coeffs)
    print("Final waste with minimize:", final_waste_SLSQP)
    print("Execution time: {:.2f} seconds".format(time_SLSQP2))
    print("---------------------------------------------------------")
else:
    print("Optimization 1 with SLSQP has not been performed")

Method set up with:  1  iterations
The total energy generated is:  967.7149999999999
The total energy corresponding to each user is:  [322.57166667 322.57166667 322.57166667]
Iteration: 0
Objective function value: 198.1343161574154
--------------------------
Iteration limit reached    (Exit mode 9)
            Current function value: 198.1343161574154
            Iterations: 1
            Function evaluations: 1366
            Gradient evaluations: 1
Constraint 1: [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  2.22044605e-16  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -1.11022302e-16  0.00000000e+00 -1.11022302e-16  0.00000000e+00
  2.22044605e-16  0.00000000e+00 -2.22044605e-16 -1.11022302e-16
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.0000000

## 1.2 Optimizing using an Approximation Method

Now, a custom method is proposed that provides a non-optimal approximation of the coefficients. The algorithm is basic and follows the ideas of the Vogel's approximation method and the northwest corner method, which are typical algorithms for optimization in transportation problems.

The steps of the algorithm are explained in detail in the main documentation.

WARNING: This method is a quick approximation. Depending on the sample, it will give acceptable or very poor results. Generally, the more disparate the sample, the poorer the results. Some print statements are included to observe how much it deviates from the row constraint. The column constraint is always met by construction.
Often, the method achieves zero energy waste to the grid but violates the row constraint.

The use of this method could help identify cases where a good approximation is achieved and avoid using the previous method, which requires significant computation.


In [25]:
def optimization_method2(E, total_e, sum_col, max_iterations = 1000, p=0.01):
    n_rows, n_cols = E.shape
    nUser = n_rows

    # Initialize coefficients matrix
    coefficients_df = pd.DataFrame(np.zeros((nUser, n_cols)), columns=[f'c{j}' for j in range(1, n_cols + 1)])

    for j in range(n_cols):  # iterate over each column

        max_coef = E[:, j] / total_e[j]  # Calculate maximum coefficients
        max_coef[max_coef > sum_col] = sum_col  # Limit coefficients to 1 if they exceed 1

        sum = 0
        
        temporary_consumptions = np.sum((total_e * coefficients_df), axis=1)
        indices = np.argsort(temporary_consumptions)  # sort consumptions from lowest to highest
        for i in indices:
            if sum + max_coef[i] > 1:
                coefficients_df.iloc[i, j] = 1 - sum
                break
            else:
                coefficients_df.iloc[i, j] = max_coef[i]
                sum += max_coef[i]
    
    print("------PRE-CORRECTION RESULTS------")
    for i in range(nUser):  
        print("The sum of the row multiplied by total generated for ", i, " is ",
              np.sum((total_e * coefficients_df.iloc[i, :])), " deviates from", consumption[i], "by ", abs(consumption[i] - np.sum((total_e * coefficients_df.iloc[i, :]))), "kWh, that is ", abs(consumption[i] - np.sum((total_e * coefficients_df.iloc[i, :])))/consumption[i]*100, "%")

    # CORRECTION PHASE
    exceed = False
    l=1
    for h in range(max_iterations):  
        error = np.zeros(nUser)

        for i in range(nUser):  
            error[i] = np.sum((total_e * coefficients_df.iloc[i, :])) - consumption[i]
        
        if np.all(error == 0) or np.all(error < 0) or np.all(error > 0):
            print("It is not possible to correct the coefficients")
            break

        indices = np.argsort(error)  # Sort errors from lowest to highest

        u2 = indices[0]
        u1 = indices[nUser - 1]

        diff = error[u2] - error[u1]
        
        if exceed:
            l += p # correction rate

        if abs(diff) < 1e-4:  # If the difference is insignificant, correction can be omitted
            print("#####            The difference is insignificant")
            break

        correction_amount = diff / n_cols * (1/l)  # Correction amount

        # Apply correction and avoid exceeding the limits [0, 1]
        coef_u1_new = coefficients_df.iloc[u1, :] + correction_amount
        coef_u2_new = coefficients_df.iloc[u2, :] - correction_amount

        # Check if the coefficients of u1 exceed the limits [0, 1], and if so, do not update them
        if (coef_u1_new > sum_col).any() or (coef_u1_new < 0).any():
            exceed = True
            # Identify the indices of the columns where coefficients exceed the limits
            invalid_indices = np.where((coef_u1_new > sum_col) | (coef_u1_new < 0))[0]

            # Maintain the original coefficients for those indices
            coef_u1_new[invalid_indices] = coefficients_df.iloc[u1, :][invalid_indices]
            coef_u2_new[invalid_indices] = coefficients_df.iloc[u2, :][invalid_indices]
        else:
            exceed = False
        # Update the coefficients
        coefficients_df.iloc[u1, :] = coef_u1_new
        coefficients_df.iloc[u2, :] = coef_u2_new

        if (h==max_iterations-2):
            print("#####            Maximum number of iterations reached", h)

    # Check prints
    print("------POST-CORRECTION RESULTS------")
    for i in range(nUser):
        print("The sum of the row multiplied by total generated for ", i, " is ",
              np.sum((total_e * coefficients_df.iloc[i, :])), " deviates from", consumption[i], "by ", abs(consumption[i] - np.sum((total_e * coefficients_df.iloc[i, :]))), "kWh, that is ", abs(consumption[i] - np.sum((total_e * coefficients_df.iloc[i, :])))/consumption[i]*100, "%")
    # Save to CSV file
    coefficients_df.to_csv('coefficients/optimization1_custom.csv', index=False)
    return coefficients_df

if optimization1 == True: 
    start_time = time.time()

    c = optimization_method2(E_non_null.to_numpy(), total_e_non_null, 1, max_iterations=10000, p=0.1)
    print(c)

    end_time = time.time()
    execution_time = end_time - start_time

    time_custom2 = execution_time
    spill_custom2 = objective_function2(c.to_numpy())

    print("Initial waste with equitable coefficients:", initial_waste_equal_coeffs)
    print("Final waste with optimization_method2: ", spill_custom2)
    print("Execution time: {:.2f} seconds" .format(time_custom2))
else:
    print("Optimization 1 with the custom method has not been performed")


------PRE-CORRECTION RESULTS------
The sum of the row multiplied by total generated for  0  is  322.46000000000004  deviates from 322.5716666666666 by  0.1116666666665651 kWh, that is  0.0346176301906755 %
The sum of the row multiplied by total generated for  1  is  322.914  deviates from 322.5716666666666 by  0.3423333333333858 kWh, that is  0.10612628718167617 %
The sum of the row multiplied by total generated for  2  is  322.341  deviates from 322.5716666666666 by  0.2306666666665933 kWh, that is  0.07150865699093018 %
#####            The difference is insignificant
------POST-CORRECTION RESULTS------
The sum of the row multiplied by total generated for  0  is  322.57166405422237  deviates from 322.5716666666666 by  2.6124442342734255e-06 kWh, that is  8.098802542918398e-07 %
The sum of the row multiplied by total generated for  1  is  322.57162553736356  deviates from 322.5716666666666 by  4.11293030424531e-05 kWh, that is  1.2750438830374575e-05 %
The sum of the row multiplied by

## 1.3 SLSQP Initializated
We will try to start the optimization method 2 of SLSQP with the obtained approximation to see if we can achieve a good result in fewer iterations.

In [26]:
if optimization1 == True:

    callback_function.iteration = 0
    c = pd.read_csv('coefficients/optimization1_custom.csv')
    c = c.to_numpy()
    c = c.flatten()

    print("Method configured with: ", max_iter, " iterations")
    start_time = time.time()

    res = minimize(objective_function2, c, bounds=bounds, constraints=constraints, method='SLSQP', callback=callback_function, options={'disp': True, 'maxiter': max_iter})

    end_time = time.time()
    execution_time = end_time - start_time

    c_optimal = res.x.reshape((n_rows, n_cols))
    # Check that constraints are met
    print("Constraint 1:", equality_constraint2_1(c_optimal))
    print("Constraint 2:", equality_constraint2_2(c_optimal))

    # Round c_optimal to 4 decimals
    c_optimal = np.round(c_optimal, 4)
    # print(c_optimal)

    coefficients_df = pd.DataFrame(c_optimal, columns=[f'c{j}' for j in range(1, n_cols + 1)])
    coefficients_df.to_csv('coefficients/optimization1_SLSQP_init.csv', index=False)

    time_SLSQP2_initialized = execution_time
    spill_SLSQP2_initialized = objective_function2(c_optimal).round(2)

    print("---------------------------------------------------------")
    print("Initial waste with equitable coefficients:", initial_waste_equal_coeffs)
    print("Final waste with minimize:", spill_SLSQP2_initialized)
    print("Execution time: {:.2f} seconds".format(time_SLSQP2_initialized))
    print("---------------------------------------------------------")

else:
    print("Optimization 1 with SLSQP initialized has not been performed")


Method configured with:  1  iterations
The total energy generated is:  967.7149999999999
The total energy corresponding to each user is:  [322.57166667 322.57166667 322.57166667]
Iteration: 0
Objective function value: 8.153226495867079
--------------------------
Iteration limit reached    (Exit mode 9)
            Current function value: 8.153226495867079
            Iterations: 1
            Function evaluations: 1366
            Gradient evaluations: 1
Constraint 1: [ 0.00000000e+00  2.22044605e-16  0.00000000e+00  6.66133815e-16
  4.44089210e-16  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6.66133815e-16  4.44089210e-16  2.22044605e-16  4.44089210e-16
 -1.11022302e-16 -5.55111512e-16  0.00000000e+00 -4.44089210e-16
 -4.44089210e-16  0.00000000e+00 -4.44089210e-16 -1.11022302e-16
 -5.55111512e-16 -6.66133815e-16 -4.44089210e-16 -4.44089210e-16
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.22044605e-16
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.000

# Optimization 2: Including the Price Variable
Now, we include a new variable, which is the price of energy. To do this, we minimize the cost of energy wasted to the grid. In other words, if we assume that energy is money, the idea is to minimize the waste of money to the grid.

Since we do not have data from Ireland and I have not found a good way to obtain and process data from Spain, the energy price will be simulated.

In [27]:
n_rows, n_cols = E_non_null.shape

energy_prices = []
for i in range(n_days):
    daily_energy_prices = np.zeros(24)
    daily_energy_prices[8:17] = np.random.uniform(0.2, 0.4, 9)  # peak hours
    daily_energy_prices[0:8] = np.random.uniform(0.04, 0.2, 8)  # off-peak hours
    daily_energy_prices[17:] = np.random.uniform(0.04, 0.2, 7)  # off-peak hours

    energy_prices = np.append(energy_prices, daily_energy_prices)

# Now delete the columns we removed at the beginning of the notebook
energy_prices = np.delete(energy_prices, null_indices)

np.savetxt('output_files/energy_prices.txt', energy_prices)
print("Energy price successfully saved in energy_prices.txt")


Energy price successfully saved in energy_prices.txt


Now with the price of energy, we can write the new problem.

$$ \min \sum_{j=1}^{nCols}\sum_{i=1}^{nUsers} \max(0, c_{ij} \cdot {totalEnergy_j} \cdot {energyPrice_j} - E_{ij} \cdot {energyPrice_j}) $$

With the constraints:
$$ \sum_{i=1}^{nUsers} c_{ij} = 1, \forall j \in nCols $$
$$ \sum_{j=1}^{nCols} E_{ij} \cdot c_{ij} \cdot {energyPrice_j} = consumo_i, \forall i \in nUsers $$

This function is very similar to the one in case 2, except that it solves the problem in euros instead of kWh. As an important note, consumption is now in euros, not kWh. But the idea is the same as in the previous case.


In [28]:
total_e_price = total_e_non_null * energy_prices
print("The total energy generation by price is: ", np.sum(total_e_price), " euros")

print("The system coefficients of each user are: ", system_coefficients)

# Each user should save the amount of money corresponding to them, i.e., the sum of the generation of energy by price by the coefficient of each user
system_coefficients = np.array(system_coefficients)
row_sum = np.sum(total_e_price) * system_coefficients
row_sum = np.array(row_sum)
print("Each user saves in a case of not consuming less than contracted: ", row_sum, " euros")

consumption = np.zeros(n_rows)
max_consumption = np.zeros(n_rows)
difference = 0
cont = 0

# Let's see how much each user can save in case someone consumes less than what they are contracted for
for i in range(n_rows):
    sum = 0
    for j in range(n_cols):
        # the maximum coefficient for user i at hour j is:
        max_coeff = E_non_null.iloc[i, j] / total_e_non_null[j]
        if max_coeff > 1:
            max_coeff = 1

        sum += max_coeff * total_e_price[j]
    print("User ", i, " can save a maximum of: ", sum, " euros")
    max_consumption[i] = sum  # in this variable I save the maximum that each user can save, if this maximum is less than what corresponds to them, then the minimum is assigned to them and the rest is distributed among the other users

# if any user(s) consume less than their corresponding share of the energy generation, then their demand is assigned as their share of the energy generation, thus the remaining energy of this user is distributed among the other users
for i in range(n_rows):
    if row_sum[i] >= max_consumption[i]:
        # save the difference
        consumption[i] = max_consumption[i]
        difference += row_sum[i] - max_consumption[i]
        cont += 1

# distribute the difference among the other users
print("Consumption in euros: ", consumption)
for i in range(n_rows):
    if consumption[i] == 0:
        consumption[i] = row_sum[i] + difference/(n_rows - cont)

print("The sum of consumption in euros is: ", np.sum(consumption))
print("The sum of energy generation in euros is: ", np.sum(row_sum))
print("Each user can save a maximum of: ", consumption, " euros")

The total energy generation by price is:  268.2168524329616  euros
The system coefficients of each user are:  [0.33333333 0.33333333 0.33333333]
Each user saves in a case of not consuming less than contracted:  [89.40561748 89.40561748 89.40561748]  euros
User  0  can save a maximum of:  94.34813644294167  euros
User  1  can save a maximum of:  216.07980906555346  euros
User  2  can save a maximum of:  120.89376546323543  euros
Consumption in euros:  [0. 0. 0.]
The sum of consumption in euros is:  268.21685243296156
The sum of energy generation in euros is:  268.21685243296156
Each user can save a maximum of:  [89.40561748 89.40561748 89.40561748]  euros


Now we are in a position to define the objective function and its constraints.

In [29]:

def objective_function3(c):
    c_matrix = c.reshape((n_rows, n_cols))
    total = 0
    for j in range(n_cols):
        for i in range(n_rows):
            total += max(0, c_matrix[i, j] * total_e_non_null[j] * energy_prices[j] - E_non_null.iloc[i, j] * energy_prices[j])
    return total

def equality_constraint3_1(c):
    return np.sum(c.reshape((n_rows, n_cols)), axis=0) - 1

def equality_constraint3_2(c):
    return np.sum((total_e_non_null * c.reshape((n_rows, n_cols))), axis=1) - consumption

print("Initial waste with equitable coefficients in euros:", objective_function3(initial_guess).round(2), " euros")
print("Each user must consume ", consumption, " in euros")

bounds = [(0, 1)] * (n_rows * n_cols)

constraints = ({
    'type': 'eq', 'fun': equality_constraint3_1},
    {'type': 'eq', 'fun': equality_constraint3_2}
)

def callback_function(xk):
    if (callback_function.iteration == 0):
        print("Execution start")
        print("The total energy generated is: ", np.sum(total_e_non_null))
        print("The total energy in euros corresponding to each user is: ", consumption)
        
    print("Iteration:", callback_function.iteration)
    print("Objective function value:", objective_function3(xk))
    print("--------------------------")
    callback_function.iteration += 1

callback_function.iteration = 0

# Initialize the coefficients randomly
initial_guess2 = np.random.rand(n_rows, n_cols)

# Normalize each column to sum to 1
initial_guess2 /= np.sum(initial_guess2, axis=0)

# Convert the matrix into a one-dimensional array
initial_guess2 = initial_guess2.flatten()

Initial waste with equitable coefficients in euros: 42.32  euros
Each user must consume  [89.40561748 89.40561748 89.40561748]  in euros


In [30]:
if optimization2 == True:
    callback_function.iteration = 0
    start_time = time.time()

    res = minimize(objective_function3, initial_guess2, bounds=bounds, constraints=constraints, method='SLSQP', callback=callback_function, options={'disp': True, 'maxiter': max_iter})

    end_time = time.time()
    execution_time = end_time - start_time
    time_SLSQP3 = execution_time

    c_optimal = res.x.reshape((n_rows, n_cols))

    # Check that the constraints are met
    print("Constraint 1:", equality_constraint3_1(c_optimal))
    print("Constraint 2:", equality_constraint3_2(c_optimal))

    # Round c_optimal to 4 decimals
    c_optimal = np.round(c_optimal, 4)
    print(c_optimal)

    coefficients_df = pd.DataFrame(c_optimal, columns=[f'c{j}' for j in range(1, n_cols + 1)])
    coefficients_df.to_csv('coefficients/optimization2_SLSQP.csv', index=False)

    initial_waste_in_euros_with_equitable_coefficients = objective_function3(initial_guess).round(2)
    final_waste_in_euros_with_minimize = objective_function3(c_optimal).round(2)

    print("---------------------------------------------------------")
    print("Initial waste in euros with equitable coefficients:", initial_waste_in_euros_with_equitable_coefficients)
    print("Final waste in euros with minimize:", final_waste_in_euros_with_minimize)
    print("---------------------------------------------------------")
        
else:
    print("Optimization 2 with SLSQP has not been performed")


Execution start
The total energy generated is:  967.7149999999999
The total energy in euros corresponding to each user is:  [89.40561748 89.40561748 89.40561748]
Iteration: 0
Objective function value: 57.34013429865431
--------------------------
Iteration limit reached    (Exit mode 9)
            Current function value: 57.34013429865431
            Iterations: 1
            Function evaluations: 1366
            Gradient evaluations: 1
Constraint 1: [-1.11022302e-16  0.00000000e+00 -2.22044605e-16  0.00000000e+00
  0.00000000e+00  2.22044605e-16  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  2.22044605e-16
 -1.11022302e-16  0.00000000e+00  2.22044605e-16  0.00000000e+00
  0.00000000e+00 -1.11022302e-16  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.0000

## 2.2 Optimizing using an Approximation Method

Now, a custom method very similar to 2.2 is proposed, but it modifies the constraints based on price instead of energy. The algorithm is basic and follows the ideas of the Vogel's approximation method and the northwest corner method, which are typical algorithms for optimization in transportation problems.

The steps of the algorithm are explained in detail in the main documentation.

WARNING: This method is a quick approximation. Depending on the sample, it will give acceptable or very poor results. Generally, the more disparate the sample, the poorer the results. Some print statements are included to observe how much it deviates from the row constraint. The column constraint is always met by construction.
Often, the method achieves zero energy waste to the grid but violates the row constraint.

The use of this method could help identify cases where a good approximation is achieved and avoid using the previous method, which requires significant computation.

In [31]:
def optimization_method3(E, total_e, energy_prices):
    n_rows, n_cols = E.shape
    nUser = n_rows

    # Initialize coefficients matrix
    coefficients_df = pd.DataFrame(np.zeros((nUser, n_cols)), columns=[f'c{j}' for j in range(1, n_cols + 1)])

    current_row = 0  # start with the first row
    for j in range(n_cols):  # start with the first column

        max_coef = E[:, j] / total_e[j]  # Calculate maximum coefficients
        max_coef[max_coef > 1] = 1  # Limit coefficients to 1 if they are greater than 1

        # Find the final row
        sum = 0
        final_row = current_row

        if j < nUser * 1:  # INITIALIZATION
            for i in range(nUser):
                if sum + max_coef[final_row] > 1:
                    coefficients_df.iloc[final_row, j] = 1 - sum
                    break
                else:
                    coefficients_df.iloc[final_row, j] = max_coef[final_row]
                    sum += max_coef[final_row]
                    final_row = (final_row + 1) % nUser
            # Update the current row
            current_row = final_row
        else:
            temp_consumption_price = np.sum((total_e * coefficients_df * energy_prices), axis=1)  # Consumption in euros
            indices = np.argsort(temp_consumption_price) 
            for i in indices:
                if sum + max_coef[i] > 1:
                    coefficients_df.iloc[i, j] = 1 - sum
                    break
                else:
                    coefficients_df.iloc[i, j] = max_coef[i]
                    sum += max_coef[i]

    print("------PRE-CORRECTION RESULTS------")
    for i in range(nUser):  # Error should decrease
        print("The sum of the row multiplied by total generated for ", i, " is ",
              np.sum((total_e * coefficients_df.iloc[i, :]) * energy_prices), " it deviates from", consumption[i], "by ", abs(consumption[i] - np.sum((total_e * coefficients_df.iloc[i, :]) * energy_prices)), "euros, that is ", abs(consumption[i] - np.sum((total_e * coefficients_df.iloc[i, :]) * energy_prices))/consumption[i]*100, "%")
            
    # CORRECTION PHASE
    for h in range(100):  # Perform 100 correction iterations, adjust this value as necessary
        error = np.zeros(nUser)
        for i in range(nUser):  # For each user
            error[i] = np.sum((total_e * coefficients_df.iloc[i, :]) * energy_prices) - consumption[i]

        if np.all(error == 0) or np.all(error > 0) or np.all(error < 0):
            print("Correction phase cannot be performed")
            break
        
        indices = np.argsort(error)

        u2 = indices[0]
        u1 = indices[-1]

        diff = error[u2] - error[u1]
        correction_amount = diff / n_cols * (1 / (h + 0.01))  # Amount of correction

        # Apply correction and avoid exceeding limits [0, 1]
        coef_u1_new = coefficients_df.iloc[u1, :] + correction_amount
        coef_u2_new = coefficients_df.iloc[u2, :] - correction_amount

        if (coef_u1_new > 1).any() or (coef_u1_new < 0).any():
            invalid_indices = np.where((coef_u1_new > 1) | (coef_u1_new < 0))[0]
            coef_u1_new[invalid_indices] = coefficients_df.iloc[u1, :][invalid_indices]
            coef_u2_new[invalid_indices] = coefficients_df.iloc[u2, :][invalid_indices]

        coefficients_df.iloc[u1, :] = coef_u1_new
        coefficients_df.iloc[u2, :] = coef_u2_new

    print("------POST-CORRECTION RESULTS------")
    for i in range(nUser):
        print("The sum of the row multiplied by total generated for ", i, " is ",
              np.sum((total_e * coefficients_df.iloc[i, :]) * energy_prices), " it deviates from", consumption[i], "by ", abs(consumption[i] - np.sum((total_e * coefficients_df.iloc[i, :]) * energy_prices)), "euros, that is ", abs(consumption[i] - np.sum((total_e * coefficients_df.iloc[i, :]) * energy_prices))/consumption[i]*100, "%")
    
    coefficients_df.to_csv('coefficients/optimization2_custom.csv', index=False)
    return coefficients_df

if optimization2 == True:
    start_time = time.time()
    c = optimization_method3(E_non_null.to_numpy(), total_e_non_null, energy_prices)
    end_time = time.time()
    execution_time = end_time - start_time
    time_custom3 = execution_time

    initial_waste_in_euros_with_equitable_coefficients = objective_function3(initial_guess).round(2)
    final_waste_custom3 = objective_function3(c.to_numpy()).round(2)

    print("Initial waste with equitable coefficients:", initial_waste_in_euros_with_equitable_coefficients)
    print("Final waste with optimization_method3: ", final_waste_custom3)
else:
    print("Optimization 2 with the custom method has not been performed")


------PRE-CORRECTION RESULTS------
The sum of the row multiplied by total generated for  0  is  89.31126098765667  it deviates from 89.40561747765386 by  0.09435648999719604 euros, that is  0.10553754077117089 %
The sum of the row multiplied by total generated for  1  is  89.64902464682282  it deviates from 89.40561747765386 by  0.243407169168961 euros, that is  0.27225042009222566 %
The sum of the row multiplied by total generated for  2  is  89.25656679848214  it deviates from 89.40561747765386 by  0.14905067917172232 euros, that is  0.16671287932100712 %
------POST-CORRECTION RESULTS------
The sum of the row multiplied by total generated for  0  is  89.39437657480738  it deviates from 89.40561747765386 by  0.011240902846481049 euros, that is  0.012572926806630035 %
The sum of the row multiplied by total generated for  1  is  89.42828688396575  it deviates from 89.40561747765386 by  0.022669406311891294 euros, that is  0.025355684521229674 %
The sum of the row multiplied by total gen

## SLSQP Initializated
We will try to start the optimization method 3 of SLSQP with the obtained approximation to see if we can achieve a good result in fewer iterations.

In [32]:
if optimization2 == True:
    # Now using the minimize method to optimize the coefficients loaded from coefficients/optimization2_custom.csv
    callback_function.iteration = 0

    # Read coefficients from coefficients/optimization2_custom.csv
    c = pd.read_csv('coefficients/optimization2_custom.csv')
    c = c.to_numpy()
    c = c.flatten()

    print("Method configured with: ", max_iter, " iterations")
    start_time = time.time()
    res = minimize(objective_function3, c, bounds=bounds, constraints=constraints, method='SLSQP', callback=callback_function, options={'disp': True, 'maxiter': max_iter})
    end_time = time.time()
    execution_time = end_time - start_time
    time_SLSQP3_initialized = execution_time

    c_optimal = res.x.reshape((n_rows, n_cols))
    # Check that the constraints are met
    print("Constraint 1:", equality_constraint3_1(c_optimal))
    print("Constraint 2:", equality_constraint3_2(c_optimal))

    # Round c_optimal to 4 decimals
    c_optimal = np.round(c_optimal, 4)
    # print(c_optimal)

    coefficients_df = pd.DataFrame(c_optimal, columns=[f'c{j}' for j in range(1, n_cols + 1)])
    coefficients_df.to_csv('coefficients/optimization2_SLSQP_init.csv', index=False)

    final_waste_SLSQP_initialized = objective_function3(c_optimal).round(2)

    print("---------------------------------------------------------")
    print("Initial waste with equitable coefficients:", initial_waste_in_euros_with_equitable_coefficients)
    print("Final waste with minimize:", final_waste_SLSQP_initialized)
    print("---------------------------------------------------------")
else:
    print("Optimization 2 with initialized SLSQP has not been performed")


Method configured with:  1  iterations
Execution start
The total energy generated is:  967.7149999999999
The total energy in euros corresponding to each user is:  [89.40561748 89.40561748 89.40561748]
Iteration: 0
Objective function value: 4.279568294286979
--------------------------
Iteration limit reached    (Exit mode 9)
            Current function value: 4.279568294286979
            Iterations: 1
            Function evaluations: 1366
            Gradient evaluations: 1
Constraint 1: [ 0.00000000e+00 -4.44089210e-16  4.44089210e-16  0.00000000e+00
  0.00000000e+00  2.22044605e-16  2.22044605e-16  4.44089210e-16
 -1.11022302e-16 -3.33066907e-16 -4.44089210e-16  0.00000000e+00
 -1.11022302e-16 -2.22044605e-16 -2.22044605e-16 -2.22044605e-16
 -2.22044605e-16  2.22044605e-16  0.00000000e+00  0.00000000e+00
 -1.11022302e-16 -2.22044605e-16 -1.11022302e-16 -2.22044605e-16
  2.22044605e-16  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  